In [1]:
import os
import random
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import save_model, Sequential
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.optimizers import Adam
from PIL import Image
from imutils import paths

In [2]:
class DataLoader(Sequence):
    def __init__(self, file_path, batch_size):
        super().__init__()
        self.path = file_path
        self.batch_size = batch_size
        classes = set([])
        for paths in file_path :
            classes.add(paths.split('/')[-2]) ## labelled by last directory's name
        self.classes = sorted(list(classes))
        
    def __len__(self):
        return (len(self.path) + self.batch_size - 1)//self.batch_size
    
    def __getitem__(self, idx):
        batch = self.path[idx*self.batch_size:(idx+1)*self.batch_size]
        data = []
        label = []
        for b in batch:
            img = Image.open(b)
            img = img.convert('RGB').resize((224,224))
            data.append(np.array(img))
            img_class = np.zeros(len(self.classes))
            img_class[self.classes.index(b.split('/')[-2])] += 1 ## one-hot encoding
            label.append(img_class)
            
        return np.array(data), np.array(label)

In [3]:
train_path = "./train"
test_path = "./test"
train_file = list(paths.list_images(train_path))
test_file = list(paths.list_images(test_path))

In [4]:
train_data = DataLoader(train_file, 16)
test_data = DataLoader(test_file, 16)

In [5]:
model = Sequential()
model.add(MobileNetV3Large(input_shape=(224, 224, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate = 0.0001), metrics=['accuracy'])

model.summary()

2022-10-11 06:40:30.987357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 06:40:31.700431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 06:40:31.701172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-11 06:40:31.719035: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

12697600/12683000 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 960)        3840      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 47040)             0         
                                                                 
 dense (Dense)               (None, 3)                 141123    
                                                                 
Total params: 3,141,315
Trainable params: 143,043
Non-trainable params: 2,998,272
_________________________________________

In [6]:
H = model.fit(train_data, validation_data=test_data, epochs=10)

Epoch 1/10


2022-10-11 06:41:14.061238: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8204
2022-10-11 06:41:23.045430: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


13/13 [==============================] - 38s 622ms/step - loss: 1.1774 - accuracy: 0.5450 - val_loss: 0.3424 - val_accuracy: 0.8889
Epoch 2/10
13/13 [==============================] - 6s 488ms/step - loss: 0.1831 - accuracy: 0.9450 - val_loss: 0.3023 - val_accuracy: 0.8889
Epoch 3/10
13/13 [==============================] - 6s 468ms/step - loss: 0.0376 - accuracy: 0.9900 - val_loss: 0.2214 - val_accuracy: 0.9259
Epoch 4/10
13/13 [==============================] - 6s 451ms/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.2288 - val_accuracy: 0.9259
Epoch 5/10
13/13 [==============================] - 6s 488ms/step - loss: 0.0126 - accuracy: 1.0000 - val_loss: 0.2788 - val_accuracy: 0.8889
Epoch 6/10
13/13 [==============================] - 6s 483ms/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.2871 - val_accuracy: 0.8889
Epoch 7/10
13/13 [==============================] - 6s 468ms/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 0.2725 - val_accuracy: 0.8889
Epoch 8/10
13/13

In [ ]:
save_model(model, "./model/1")